In [8]:
%pwd

'd:\\sign_language_conversion'

In [9]:
import os 
os.chdir("../")

In [10]:
%pwd

'd:\\'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataPickle:
    root_dir: Path
    data_pickle: Path

In [12]:
from sign_language_conversion.constants import *
from sign_language_conversion.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_creation(self) -> DataPickle:
        config = self.config.pickle_dataset
        create_directories([config.root_dir])

        # Check if 'no_of_classes' exists in params
        if 'no_of_classes' not in self.params:
            raise KeyError("The key 'no_of_classes' is missing from params file.")

        data_pickle_config = DataPickle(
            root_dir=config.root_dir,
            data_pickle=config.data_pickle
        )
        return data_pickle_config

In [15]:
import os
import cv2
from sign_language_conversion import logger
from sign_language_conversion.utils.common import get_size

In [16]:
import os
import cv2
import pickle
import mediapipe as mp

class DataFetcher:
    def __init__(self, config):
        self.config = config
    
    def datapick(self) -> str:

        try: 
            mp_hands = mp.solutions.hands

            hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

            data = []
            labels = []
            for dir_ in os.listdir(self.data_dir):
                for img_path in os.listdir(os.path.join(self.config.data_creation, dir_)):
                    data_aux = []

                    x_ = []
                    y_ = []

                    img = cv2.imread(os.path.join(self.data_dir, dir_, img_path))
                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    results = hands.process(img_rgb)
                    if results.multi_hand_landmarks:
                        for hand_landmarks in results.multi_hand_landmarks:
                            for i in range(len(hand_landmarks.landmark)):
                                x = hand_landmarks.landmark[i].x
                                y = hand_landmarks.landmark[i].y

                                x_.append(x)
                                y_.append(y)

                            for i in range(len(hand_landmarks.landmark)):
                                x = hand_landmarks.landmark[i].x
                                y = hand_landmarks.landmark[i].y
                                data_aux.append(x - min(x_))
                                data_aux.append(y - min(y_))

                        data.append(data_aux)
                        labels.append(dir_)

            with open('artifacts/data_pickle/data.pickle', 'wb') as f:
                pickle.dump({'data': data, 'labels': labels}, f)
            
            return 'data.pickle'
        
        except Exception as e:
            raise e

In [17]:
try:
    config = ConfigurationManager()
    data_creation_config = config.get_creation()
    data_creator = DataFetcher(config=data_pickle_config)
    data_creator.datapick()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'